# Inertia Tensor to Point Mass Reconstruction

This is one of those little interesting problems that's easy to specify, but becomes a bit of an interesting head-scratcher when you get into it.


## Problem Statement

Say you're given a set of mass properties, as would be specified in the AeroSandbox `MassProperties` class. Specifically, you have the following information:

- A mass $m$
- A center of gravity position $\vec{r}_{\text{CG}}$
- A 3x3 inertia tensor $\mathbf{I}$. This is a matrix of the form:

$$
\mathbf{I} = \begin{bmatrix}
I_{xx} & I_{xy} & I_{xz} \\
I_{yx} & I_{yy} & I_{yz} \\
I_{zx} & I_{zy} & I_{zz}
\end{bmatrix}
$$

where $I_{xx}$ is the moment of inertia about the $x$-axis, $I_{xy}$ is the product of inertia about the $x$- and $y$-axes, etc.

Note that this inertia tensor is symmetric, so you really have 6 independent parameters from $\mathbf{I}$, not 9. You then have 3 more independent parameters from the center of gravity position, and 1 more independent parameter from the mass. This yields a total of 10 independent parameters.

Now, your goal is to identify a set of multiple point masses that, when combined, have the same mass properties as the original mass properties. In other words, you want to find a set of point masses $\{m_i, \vec{r}_{i}\}$ such that:

$$
\sum_{i=1}^N m_i = m
$$

$$
\sum_{i=1}^N m_i \vec{r}_{i} = m \vec{r}_{\text{CG}}
$$

$$
\sum_{i=1}^N m_i \left(\vec{r}_{i} - \vec{r}_{\text{CG}}\right) \left(\vec{r}_{i} - \vec{r}_{\text{CG}}\right)^T = \mathbf{I}
$$

where $N$ is the number of point masses.

Call this set of point masses a **constellation**.

## Numerical Solution

By inspection, we know a constellation must have, at minimum, 3 points. This is because we have 10 equations, and each point gets us 4 degrees of freedom. So, $\left\lceil \frac{10}{4} \right\rceil = 3$.


In actuality, we'll need more than 3, though - I think we'll need 4. This is because 3 points in 3D plane defines a plane, so the constellation is a flat object in 3D space. This flat object can't have any product of inertia involving the axis normal to the plane, so it can't reconstruct an arbitrary inertia tensor. So, intuitively I think we'll need 4. This also makes sense because 4 points in 3D space defines the vertices of a tetrahedron, which is the simplest possible 3D shape.

Before doing any derivations, let's check this with a numerical example to see if this will work:

In [1]:
import aerosandbox as asb
import aerosandbox.numpy as np

### Define a random mass properties object

r = lambda: np.random.randn()

valid_mass_props = False
while not valid_mass_props:
    mass_props = asb.MassProperties(
        mass=r(),
        x_cg=r(), y_cg=r(), z_cg=r(),
        Ixx=r(), Iyy=r(), Izz=r(),
        Ixy=r(), Iyz=r(), Ixz=r(),
    )
    valid_mass_props = mass_props.is_physically_possible()  # adds a bunch of checks

mass_props

MassProperties instance:
                 Mass :      1.66258
    Center of Gravity : (    0.288889,     0.544547,      0.40036)
       Inertia Tensor : 
            (about CG)  [     1.17821,   -0.0276847,    -0.207154]
                        [  -0.0276847,     0.814315,    -0.346755]
                        [   -0.207154,    -0.346755,      0.54484]

In [2]:
# Now, reconstruct the mass properties from a constellation of N point masses.

opti = asb.Opti()  # We'll treat this as an optimization problem.

# Define a set of point masses, each with identical mass
N = 4

point_masses = [
    asb.MassProperties(  # Guess random points for the constellation
        mass=mass_props.mass / N,
        x_cg=opti.variable(init_guess=mass_props.x_cg + r()),
        y_cg=opti.variable(init_guess=mass_props.y_cg + r()),
        z_cg=opti.variable(init_guess=mass_props.z_cg + r()),
    )
    for i in range(N)
]

mass_props_reconstructed = sum(point_masses)

# Add constraints
opti.subject_to(mass_props_reconstructed.x_cg == mass_props.x_cg)
opti.subject_to(mass_props_reconstructed.y_cg == mass_props.y_cg)
opti.subject_to(mass_props_reconstructed.z_cg == mass_props.z_cg)
opti.subject_to(mass_props_reconstructed.Ixx == mass_props.Ixx)
opti.subject_to(mass_props_reconstructed.Iyy == mass_props.Iyy)
opti.subject_to(mass_props_reconstructed.Izz == mass_props.Izz)
opti.subject_to(mass_props_reconstructed.Ixy == mass_props.Ixy)
opti.subject_to(mass_props_reconstructed.Iyz == mass_props.Iyz)
opti.subject_to(mass_props_reconstructed.Ixz == mass_props.Ixz)

sol = opti.solve()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       68
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       78

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        9
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 0

So, this seems to work just fine. Let's plot where the resulting points are:

In [3]:
point_masses = sol(point_masses)

%matplotlib notebook
import matplotlib.pyplot as plt
import aerosandbox.tools.pretty_plots as p

fig, ax = p.figure3d()

for point_mass in point_masses:
    ax.scatter(
        point_mass.x_cg,
        point_mass.y_cg,
        point_mass.z_cg,
        color="k",
        s=100,
    )

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

p.show_plot(dpi=160)


<IPython.core.display.Javascript object>


Let's brute-force check that this works for a bunch of random mass properties:

In [4]:
from typing import List


def solve_problem() -> List[asb.MassProperties]:
    valid_mass_props = False
    while not valid_mass_props:
        mass_props = asb.MassProperties(
            mass=r(),
            x_cg=r(), y_cg=r(), z_cg=r(),
            Ixx=r(), Iyy=r(), Izz=r(),
            Ixy=r(), Iyz=r(), Ixz=r(),
        )
        valid_mass_props = mass_props.is_physically_possible()  # adds a bunch of checks
    # Now, reconstruct the mass properties from a constellation of N point masses.

    opti = asb.Opti()  # We'll treat this as an optimization problem.

    # Define a set of point masses, each with identical mass
    N = 4

    point_masses = [
        asb.MassProperties(  # Guess random points for the constellation
            mass=mass_props.mass / N,
            x_cg=opti.variable(init_guess=mass_props.x_cg + r()),
            y_cg=opti.variable(init_guess=mass_props.y_cg + r()),
            z_cg=opti.variable(init_guess=mass_props.z_cg + r()),
        )
        for i in range(N)
    ]

    mass_props_reconstructed = sum(point_masses)

    # Add constraints
    opti.subject_to(mass_props_reconstructed.x_cg == mass_props.x_cg)
    opti.subject_to(mass_props_reconstructed.y_cg == mass_props.y_cg)
    opti.subject_to(mass_props_reconstructed.z_cg == mass_props.z_cg)
    opti.subject_to(mass_props_reconstructed.Ixx == mass_props.Ixx)
    opti.subject_to(mass_props_reconstructed.Iyy == mass_props.Iyy)
    opti.subject_to(mass_props_reconstructed.Izz == mass_props.Izz)
    opti.subject_to(mass_props_reconstructed.Ixy == mass_props.Ixy)
    opti.subject_to(mass_props_reconstructed.Iyz == mass_props.Iyz)
    opti.subject_to(mass_props_reconstructed.Ixz == mass_props.Ixz)

    return opti.solve(verbose=False)(point_masses)


from tqdm import tqdm

solutions = [
    solve_problem()
    for i in tqdm(range(100))
]

100%|██████████| 100/100 [00:10<00:00,  9.27it/s]


Now, what if we solve it a bunch of times for the same MassProperties object, and we see how the point masses move around? (Since we suspect the problem is underconstrained.)

In [5]:
from typing import List

valid_mass_props = False
while not valid_mass_props:
    mass_props = asb.MassProperties(
        mass=r(),
        x_cg=r(), y_cg=r(), z_cg=r(),
        Ixx=r(), Iyy=r(), Izz=r(),
        Ixy=r(), Iyz=r(), Ixz=r(),
    )
    valid_mass_props = mass_props.is_physically_possible()  # adds a bunch of checks


# Now, reconstruct the mass properties from a constellation of N point masses.

def solve_problem() -> List[asb.MassProperties]:
    opti = asb.Opti()  # We'll treat this as an optimization problem.

    # Define a set of point masses, each with identical mass
    N = 4

    point_masses = [
        asb.MassProperties(  # Guess random points for the constellation
            mass=mass_props.mass / N,
            x_cg=opti.variable(init_guess=mass_props.x_cg + r()),
            y_cg=opti.variable(init_guess=mass_props.y_cg + r()),
            z_cg=opti.variable(init_guess=mass_props.z_cg + r()),
        )
        for i in range(N)
    ]

    mass_props_reconstructed = sum(point_masses)

    # Add constraints
    opti.subject_to(mass_props_reconstructed.x_cg == mass_props.x_cg)
    opti.subject_to(mass_props_reconstructed.y_cg == mass_props.y_cg)
    opti.subject_to(mass_props_reconstructed.z_cg == mass_props.z_cg)
    opti.subject_to(mass_props_reconstructed.Ixx == mass_props.Ixx)
    opti.subject_to(mass_props_reconstructed.Iyy == mass_props.Iyy)
    opti.subject_to(mass_props_reconstructed.Izz == mass_props.Izz)
    opti.subject_to(mass_props_reconstructed.Ixy == mass_props.Ixy)
    opti.subject_to(mass_props_reconstructed.Iyz == mass_props.Iyz)
    opti.subject_to(mass_props_reconstructed.Ixz == mass_props.Ixz)

    return opti.solve(verbose=False)(point_masses)


from tqdm import tqdm

solutions = [
    solve_problem()
    for i in tqdm(range(100))
]

100%|██████████| 100/100 [00:05<00:00, 19.52it/s]


In [6]:
fig, ax = p.figure3d()

for solution in solutions:
    for point_mass in solution:
        ax.scatter(
            point_mass.x_cg,
            point_mass.y_cg,
            point_mass.z_cg,
            color="k",
            s=10,
            alpha=0.1,
        )

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

p.show_plot(dpi=160)

<IPython.core.display.Javascript object>

Super cool. Some observations:

1. The point masses are moving around, so the problem is indeed underconstrained.
2. The point masses are drawn from the surface of an ellipsoid.

This is all very cool. We observe that, without loss of generality, we could arbitrarily make exactly 3 constraints on any of the individual point masses (thus, pinning down exactly which points would be chosen). Let's try that, and see the result.

In [7]:
valid_mass_props = False
while not valid_mass_props:
    mass_props = asb.MassProperties(
        mass=r(),
        x_cg=r(), y_cg=r(), z_cg=r(),
        Ixx=r(), Iyy=r(), Izz=r(),
        Ixy=r(), Iyz=r(), Ixz=r(),
    )
    valid_mass_props = mass_props.is_physically_possible()  # adds a bunch of checks


# Now, reconstruct the mass properties from a constellation of N point masses.

def solve_problem() -> List[asb.MassProperties]:
    opti = asb.Opti()  # We'll treat this as an optimization problem.

    # Define a set of point masses, each with identical mass
    N = 4

    point_masses = [
        asb.MassProperties(  # Guess random points for the constellation
            mass=mass_props.mass / N,
            x_cg=opti.variable(init_guess=mass_props.x_cg + r()),
            y_cg=opti.variable(init_guess=mass_props.y_cg + r()),
            z_cg=opti.variable(init_guess=mass_props.z_cg + r()),
        )
        for i in range(N)
    ]

    mass_props_reconstructed = sum(point_masses)

    # Add constraints
    opti.subject_to(mass_props_reconstructed.x_cg == mass_props.x_cg)
    opti.subject_to(mass_props_reconstructed.y_cg == mass_props.y_cg)
    opti.subject_to(mass_props_reconstructed.z_cg == mass_props.z_cg)
    opti.subject_to(mass_props_reconstructed.Ixx == mass_props.Ixx)
    opti.subject_to(mass_props_reconstructed.Iyy == mass_props.Iyy)
    opti.subject_to(mass_props_reconstructed.Izz == mass_props.Izz)
    opti.subject_to(mass_props_reconstructed.Ixy == mass_props.Ixy)
    opti.subject_to(mass_props_reconstructed.Iyz == mass_props.Iyz)
    opti.subject_to(mass_props_reconstructed.Ixz == mass_props.Ixz)

    opti.subject_to(point_masses[0].y_cg == mass_props.y_cg)
    opti.subject_to(point_masses[0].z_cg == mass_props.z_cg)
    opti.subject_to(point_masses[1].y_cg == mass_props.y_cg)

    return opti.solve(verbose=False)(point_masses)


from tqdm import tqdm

solutions = [
    solve_problem()
    for i in tqdm(range(100))
]

100%|██████████| 100/100 [00:05<00:00, 17.51it/s]


In [8]:
fig, ax = p.figure3d()

for solution in solutions:
    for point_mass in solution:
        ax.scatter(
            point_mass.x_cg,
            point_mass.y_cg,
            point_mass.z_cg,
            color="k",
            s=10,
            alpha=0.1,
        )

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

p.show_plot(dpi=160)

<IPython.core.display.Javascript object>

Interesting. Now, it looks like there are only a finite number of solutions, which are all essentially mirrors of each other (presumably about the principal planes).

Let's see if we can pin it down to just a single solution, using inequality constraints and better initial guesses.

In [9]:
valid_mass_props = False
while not valid_mass_props:
    mass_props = asb.MassProperties(
        mass=r(),
        x_cg=r(), y_cg=r(), z_cg=r(),
        Ixx=r(), Iyy=r(), Izz=r(),
        Ixy=r(), Iyz=r(), Ixz=r(),
    )
    valid_mass_props = mass_props.is_physically_possible()  # adds a bunch of checks


# Now, reconstruct the mass properties from a constellation of N point masses.

def solve_problem() -> List[asb.MassProperties]:
    opti = asb.Opti()  # We'll treat this as an optimization problem.

    # Define a set of point masses, each with identical mass

    approximate_radius = (
                                 mass_props.Ixx + mass_props.Iyy + mass_props.Izz
                         ) ** 0.5 / mass_props.mass

    point_masses = [
        asb.MassProperties(
            mass=mass_props.mass / 4,
            x_cg=opti.variable(init_guess=mass_props.x_cg - approximate_radius, scale=approximate_radius),
            y_cg=opti.variable(init_guess=mass_props.y_cg, scale=approximate_radius),
            z_cg=opti.variable(init_guess=mass_props.z_cg, scale=approximate_radius),
        ),
        asb.MassProperties(
            mass=mass_props.mass / 4,
            x_cg=opti.variable(init_guess=mass_props.x_cg, scale=approximate_radius),
            y_cg=opti.variable(init_guess=mass_props.y_cg, scale=approximate_radius),
            z_cg=opti.variable(init_guess=mass_props.z_cg + approximate_radius, scale=approximate_radius),
        ),
        asb.MassProperties(
            mass=mass_props.mass / 4,
            x_cg=opti.variable(init_guess=mass_props.x_cg, scale=approximate_radius),
            y_cg=opti.variable(init_guess=mass_props.y_cg, scale=approximate_radius),
            z_cg=opti.variable(init_guess=mass_props.z_cg - approximate_radius, scale=approximate_radius),
        ),
        asb.MassProperties(
            mass=mass_props.mass / 4,
            x_cg=opti.variable(init_guess=mass_props.x_cg, scale=approximate_radius),
            y_cg=opti.variable(init_guess=mass_props.y_cg + approximate_radius, scale=approximate_radius),
            z_cg=opti.variable(init_guess=mass_props.z_cg, scale=approximate_radius),
        ),
    ]

    mass_props_reconstructed = sum(point_masses)

    # Add constraints
    opti.subject_to(mass_props_reconstructed.x_cg == mass_props.x_cg)
    opti.subject_to(mass_props_reconstructed.y_cg == mass_props.y_cg)
    opti.subject_to(mass_props_reconstructed.z_cg == mass_props.z_cg)
    opti.subject_to(mass_props_reconstructed.Ixx == mass_props.Ixx)
    opti.subject_to(mass_props_reconstructed.Iyy == mass_props.Iyy)
    opti.subject_to(mass_props_reconstructed.Izz == mass_props.Izz)
    opti.subject_to(mass_props_reconstructed.Ixy == mass_props.Ixy)
    opti.subject_to(mass_props_reconstructed.Iyz == mass_props.Iyz)
    opti.subject_to(mass_props_reconstructed.Ixz == mass_props.Ixz)

    opti.subject_to(point_masses[0].y_cg == mass_props.y_cg)
    opti.subject_to(point_masses[0].z_cg == mass_props.z_cg)
    opti.subject_to(point_masses[1].y_cg == mass_props.y_cg)

    opti.subject_to(point_masses[0].x_cg < point_masses[1].x_cg)

    return opti.solve(verbose=False)(point_masses)


from tqdm import tqdm

solutions = [
    solve_problem()
    for i in tqdm(range(100))
]

100%|██████████| 100/100 [00:06<00:00, 16.30it/s]


In [10]:
fig, ax = p.figure3d()

for solution in solutions:
    for point_mass in solution:
        ax.scatter(
            point_mass.x_cg,
            point_mass.y_cg,
            point_mass.z_cg,
            color="k",
            s=10,
            alpha=0.1,
        )

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

p.show_plot(dpi=160)

<IPython.core.display.Javascript object>

Nice! That seems to pin it down to one unique solution.